# Jupyter notebook for CalLat gA project
<table style="width:100%; float:left">
<tr>
<td style="text-align:left"> <img src="./callat_logo_awl_v2_transparent.png" alt="CalLat logo" style="width: 25%;"/> </td>
<td style="text-align:left"> <img src="./feyn_diagram.png" alt="gA cartoon" style="width: 25%;"/> </td>
</tr>
</table>

### Import libraries

In [1]:
import pandas as pd
pd.options.display.max_rows = 12
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import lsqfit
import gvar as gv
import ga_chipt_lib as xlib
print("pandas version:", pd.__version__)
print("numpy  version:", np.__version__)
print("mpl    version:", mpl.__version__)
print("lsqfit version:", lsqfit.__version__)
print("gvar   version:", gv.__version__)

pandas version: 0.20.1
numpy  version: 1.12.1
mpl    version: 2.0.2
lsqfit version: 9.1.3
gvar   version: 8.3.2


### Define analysis parameters

In [2]:
switches = dict()
# Ensembles used in extrapolation
switches['ensembles'] = ['l1648f211b580m013m065m838','l2448f211b580m0064m0640m828','l3248f211b580m00235m0647m831',
                'l2464f211b600m0170m0509m635','l2464f211b600m0130m0509m635','l2464f211b600m0102m0509m635','l2464f211b600m00507m0507m628','l3264f211b600m00507m0507m628','l4064f211b600m00507m0507m628', 'l4864f211b600m00184m0507m628',
                'l3296f211b630m0074m037m440','l4896f211b630m00363m0363m430']

switches['ansatz'] = dict()
switches['ansatz']['type'] = 'xpt' # Type of fit: 'Xpt' or 'taylor'
switches['ansatz']['truncation'] = 4 # Order to truncate series
switches['ansatz']['FV'] = True # True turns on NLO FV correction
switches['ansatz']['xsb'] = False # True turns on O(a) discretization
switches['ansatz']['alpha'] = False # True turns on O(alpha_s a^2) discretization

### Define priors and PDG values

In [3]:
priors = dict()
# Xpt priors
priors['xpt'] = dict()
priors['xpt']['g0'] = gv.gvar(1.0, 1.0) # LO LEC
priors['xpt']['a1'] = gv.gvar(0.0, 1E-3) # DWF order a discretization
priors['xpt']['c2'] = gv.gvar(0.0, 50.0) # NLO counterterm epi^2
priors['xpt']['c3'] = gv.gvar(0.0, 50.0) # NNLO LEC epi^3
priors['xpt']['a2'] = gv.gvar(0.0, 50.0) # NLO a^2
priors['xpt']['s2'] = gv.gvar(0.0, 1.0) # NLO alpha_s a^2
priors['xpt']['a4'] = gv.gvar(0.0, 1.0) # NNNLO a^4
priors['xpt']['b4'] = gv.gvar(0.0, 1.0) # NNNLO a^2 epi^2
priors['xpt']['c4'] = gv.gvar(0.0, 10.0) # NNNLO epi^4
# taylor priors
priors['taylor'] = dict()
priors['taylor']['c0'] = gv.gvar(1.0, 10.0) # constant
priors['taylor']['c2'] = gv.gvar(0.0, 10.0) # epi^2
priors['taylor']['a2'] = gv.gvar(0.0, 10.0) # a^2
priors['taylor']['c4'] = gv.gvar(0.0, 10.0) # epi^4
priors['taylor']['a4'] = gv.gvar(0.0, 1.0) # a^4
priors['taylor']['b4'] = gv.gvar(0.0, 1.0) # a^2 epi^2

# Physical parameters from PDG
phys_params = dict()
# http://pdg.lbl.gov/2016/tables/rpp2016-tab-mesons-light.pdf
phys_params['mpi'] = gv.gvar(139.57018, 0.00035) # mpi +/- [MeV]
# http://pdg.lbl.gov/2016/reviews/rpp2016-rev-pseudoscalar-meson-decay-cons.pdf
phys_params['fpi'] = gv.gvar(130.2, 1.7) # fpi + ['MeV']

### Import data

In [4]:
# import correlator bootstraps
gadf = pd.read_csv('./github_ga_v1.csv')
gadf.groupby('ensemble').describe()[['ga','epi']]

ga                                          \
                               count      mean       std       min       25%   
ensemble                                                                       
l1648f211b580m013m065m838     5001.0  1.215922  0.011742  1.180243  1.207896   
l2448f211b580m0064m0640m828   5001.0  1.234081  0.011274  1.194841  1.226436   
l2464f211b600m00507m0507m628  5001.0  1.271018  0.027648  1.175531  1.252519   
l2464f211b600m0102m0509m635   5001.0  1.236402  0.007431  1.204905  1.231601   
l2464f211b600m0130m0509m635   5001.0  1.230357  0.003939  1.214195  1.227834   
l2464f211b600m0170m0509m635   5001.0  1.231317  0.005364  1.203449  1.228409   
l3248f211b580m00235m0647m831  5001.0  1.262219  0.072413  1.071303  1.213338   
l3264f211b600m00507m0507m628  5001.0  1.258742  0.014600  1.197767  1.248760   
l3296f211b630m0074m037m440    5001.0  1.256943  0.014634  1.210480  1.246890   
l4064f211b600m00507m0507m628  5001.0  1.251906  0.021038  1.172414  1.238134   
l4864f211b600m00184m0507m628  5001.0  1.289305  0.029890  1.180580  1.268853   
l4896f211b630m00363m0363m430  5001.0  1.252071  0.008538  1.209421  1.246589   

                                                               epi            \
                                   50%       75%       max   count      mean   
ensemble                                                                       
l1648f211b580m013m065m838     1.214899  1.222887  1.281075  5001.0  0.249570   
l2448f211b580m0064m0640m828   1.233937  1.241649  1.272587  5001.0  0.181232   
l2464f211b600m00507m0507m628  1.270476  1.289662  1.380602  5001.0  0.184187   
l2464f211b600m0102m0509m635   1.236636  1.241467  1.261416  5001.0  0.244215   
l2464f211b600m0130m0509m635   1.230354  1.232892  1.245310  5001.0  0.271400   
l2464f211b600m0170m0509m635   1.231706  1.234309  1.259442  5001.0  0.298302   
l3248f211b580m00235m0647m831  1.254129  1.300221  1.847861  5001.0  0.113400   
l3264f211b600m00507m0507m628  1.258865  1.268513  1.315906  5001.0  0.182205   
l3296f211b630m0074m037m440    1.256645  1.266344  1.321978  5001.0  0.246186   
l4064f211b600m00507m0507m628  1.251229  1.264230  1.339749  5001.0  0.181555   
l4864f211b600m00184m0507m628  1.288847  1.309447  1.394666  5001.0  0.113472   
l4896f211b630m00363m0363m430  1.251934  1.257544  1.288477  5001.0  0.181974   

                                                                      \
                                   std       min       25%       50%   
ensemble                                                               
l1648f211b580m013m065m838     0.000360  0.248304  0.249325  0.249569   
l2448f211b580m0064m0640m828   0.000361  0.180032  0.180986  0.181232   
l2464f211b600m00507m0507m628  0.000567  0.182176  0.183812  0.184181   
l2464f211b600m0102m0509m635   0.000532  0.242349  0.243861  0.244211   
l2464f211b600m0130m0509m635   0.000355  0.269959  0.271164  0.271398   
l2464f211b600m0170m0509m635   0.000416  0.297149  0.298031  0.298278   
l3248f211b580m00235m0647m831  0.000735  0.108955  0.112971  0.113465   
l3264f211b600m00507m0507m628  0.000423  0.180769  0.181916  0.182197   
l3296f211b630m0074m037m440    0.000440  0.244618  0.245891  0.246187   
l4064f211b600m00507m0507m628  0.000440  0.180092  0.181268  0.181547   
l4864f211b600m00184m0507m628  0.000504  0.109478  0.113167  0.113512   
l4896f211b630m00363m0363m430  0.000374  0.179307  0.181720  0.181973   

                                                  
                                   75%       max  
ensemble                                          
l1648f211b580m013m065m838     0.249802  0.251079  
l2448f211b580m0064m0640m828   0.181475  0.183116  
l2464f211b600m00507m0507m628  0.184557  0.187149  
l2464f211b600m0102m0509m635   0.244581  0.246738  
l2464f211b600m0130m0509m635   0.271637  0.272696  
l2464f211b600m0170m0509m635   0.298529  0.301191  
l3248f211b580m00235m0647m831  0.113904  0.116193  
l3264f211b600m00507m0507m628  0.182498  0

In [5]:
# import HISQ parameters
hqdf = pd.read_csv('./hisq_params.csv')
hqdf

,ensemble,aw0_mean,aw0_sdev,alfs
0,l1648f211b580m013m065m838,0.8804,0.0003,0.58801
1,l2448f211b580m0064m0640m828,0.8804,0.0003,0.58801
2,l3248f211b580m00235m0647m831,0.8804,0.0003,0.58801
3,l2464f211b600m0170m0509m635,0.7036,0.0005,0.53796
4,l2464f211b600m0130m0509m635,0.7036,0.0005,0.53796
5,l2464f211b600m0102m0509m635,0.7036,0.0005,0.53796
6,l2464f211b600m00507m0507m628,0.7036,0.0005,0.53796
7,l3264f211b600m00507m0507m628,0.7036,0.0005,0.53796
8,l4064f211b600m00507m0507m628,0.7036,0.0005,0.53796
9,l4864f211b600m00184m0507m628,0.7036,0.0005,0.53796


### Format data for analysis

In [6]:
data = xlib.format_data(switches, gadf, hqdf)

### Chiral-continuum fit

In [7]:
result = xlib.fit_data(switches, priors, data, phys_params)
result

Least Square Fit:
  chi2/dof [dof] = 0.43 [12]    Q = 0.95    logGBF = 17.807

Parameters:
          epi 0      0.24955 (36)       [ 0.24957 (36) ]  
              1      0.18123 (36)       [ 0.18123 (36) ]  
              2      0.11347 (74)       [ 0.11346 (75) ]  
              3      0.29828 (39)       [ 0.29828 (39) ]  
              4      0.27140 (36)       [ 0.27140 (36) ]  
              5      0.24420 (52)       [ 0.24421 (52) ]  
              6      0.18419 (57)       [ 0.18418 (58) ]  
              7      0.18219 (44)       [ 0.18220 (44) ]  
              8      0.18155 (42)       [ 0.18155 (42) ]  
              9      0.11351 (52)       [ 0.11351 (52) ]  
             10      0.24618 (44)       [ 0.24619 (44) ]  
             11      0.18198 (37)       [ 0.18197 (37) ]  
          aw0 0      0.88040 (30)       [ 0.88040 (30) ]  
              1      0.88040 (30)       [ 0.88040 (30) ]  
              2      0.88040 (30)       [ 0.88040 (30) ]  
              3      0.7

NameError: name 'eval_phys' is not defined

# Setup for Python environment
### Download Anaconda and install 
Download [Anaconda](https://www.continuum.io/downloads) and follow installation instructions.

### Create Python environment with Anaconda
```bash
conda create --name callat_ga python=3 conda
source activate callat_ga
pip install gvar
pip install lsqfit```

Key libraries from [gplepage GitHub](https://github.com/gplepage).
* `gvar` version 8.0
* `lsqfit` version 8.1

Exit conda environment with
```bash
source deactivate
```